# Spark MLLib Example: Clustering

Let's look at a clustering example in Spark MLLib.

Here, we are going to load the mtcars dataset. This has some stats on different models of cars.  Here, we will load the CSV file as a spark dataframe, and view it.

This dataset contains some statistics on 1974 Cars from Motor Trends

Here are the columns:
* name   - name of the car
*  mpg   - Miles/(US) gallon                        
*  cyl   - Number of cylinders                      
*  disp  - Displacement (cu.in.)                    
*  hp    - Gross horsepower                         
*  drat  - Rear axle ratio            

Are there any natural clusters you can identify from this data?

We are going to use **MPG and CYL** attributes to cluster.

You can also download and view the raw data in Excel : [cars.csv](/data/cars/mtcars_header.csv)

<img src="../../assets/images/6.1-cars2.png" style="border: 5px solid grey; max-width:100%;" />

In [ ]:
# initialize Spark Session
import os
import sys
top_dir = os.path.abspath(os.path.join(os.getcwd(), "../../"))
if top_dir not in sys.path:
    sys.path.append(top_dir)

from init_spark import init_spark
spark = init_spark()
sc = spark.sparkContext

## Step 1 : Load Data

In [ ]:
dataset = spark.read.csv("../data/cars/mtcars_header.csv", header=True, inferSchema=True)

In [ ]:
## TODO : print schema
## Hint : printSchema()
dataset.printSchema()

In [ ]:
## TODO : display the data
## Hint : show
dataset.show()

## Step 2 : Extract data
We only care about 'model', 'mpg' and 'cyl' columns

In [ ]:
dataset2 = dataset.select(["model", "mpg", "cyl"])
dataset2.show()

## Step 3 : Creating Vectors

Now that we have ourselves a dataframe, let's work on turning it into vectors.  We're going to vectorize 2 columns:

1. MPG
2. Number of cylinders.

What we'll do, is we'll use the VectorAssembler class to create a new column by the name of features. This will be a Vector.

In [ ]:
from pyspark.ml.feature import VectorAssembler

## TODO : create featureVector with 'mpg' and 'cyl'
## Hint :  inputCols=['mpg', 'cyl']
assembler = VectorAssembler(inputCols=["???", "???"], outputCol="features")
featureVector = assembler.transform(dataset2)
featureVector.show()

### WSSSE calcluations

In [ ]:
# from : https://spark.apache.org/docs/3.0.0-preview/mllib-clustering.html

# # Build the model (cluster the data)
# clusters = KMeans.train(parsedData, 2, maxIterations=10, initializationMode="random")

# # Evaluate clustering by computing Within Set Sum of Squared Errors
# def error(point):
#     center = clusters.centers[clusters.predict(point)]
#     return sqrt(sum([x**2 for x in (point - center)]))

# WSSSE = parsedData.map(lambda point: error(point)).reduce(lambda x, y: x + y)

import numpy as np

def calculate_wsse (predictions, model):
    centers = model.clusterCenters()
    
    wssse = 0
    for i, row in enumerate(predictions.collect()):
        feature = row['features']
        prediction = row['prediction']
        center = centers[prediction]
        diff = feature - center
        error = np.sqrt(sum([x**2 for x in diff]))
        wssse += error
        # print ('row={}, feature : {} , prediction : {} , center : {},  diff : {}, error : {},  wssse={}'.
        #      format(i, feature, prediction, center, diff, error, wssse))
        
    return wssse


## Step 4 : Running Kmeans

Now it's time to run kmeans on the resultant dataframe. We don't know what value of k to use, so let's just start with k=2.  This means we will cluster into two groups.

We will fit a model to the data, and then train it.

### 4.1 - Run KMeans

In [ ]:
%%time 

from pyspark.ml.clustering import KMeans

k = 2
kmeans = KMeans().setK(k).setMaxIter(10)
model = kmeans.fit(featureVector)


### 4.2 -  Predict

Notice what we have here.  We have two clusters. One is smaller, fuel efficient cars like the Fiat and the Corolla (remember, we cluster on two variables only: mpg and cyl).  The other is for basically all other cars.  Probably, we can get better results here with a differnet value of k.

In [ ]:
predictions = model.transform(featureVector)
predictions.orderBy(['prediction', 'mpg']).show(32)

### 4.3 - Calculate the kmeans WSSSE and Silhouette score

**WSSSE** is an error metric that calcluates how far away points is from their center.  The less the better.

**Silhouette** score is used to evaluate the quality of clusters created using clustering algorithms such as K-Means.  
Silhoutte score ranges from -1 to +1.  
The silhouette score of 1 means that the clusters are very dense and nicely separated.  
The score of 0 means that clusters are overlapping.  
The score of less than 0 means that data belonging to clusters may be wrong/incorrect.

In [ ]:
from pyspark.ml.evaluation import ClusteringEvaluator

wssse = calculate_wsse(predictions, model)

evaluator = ClusteringEvaluator()
silhouette = evaluator.evaluate(predictions)
print ("k={}, wssse={},  silhouette_score={}".format(k, wssse, silhouette))

## Step 5 : Adjust K

In [ ]:
k = 3
kmeans = KMeans().setK(k).setMaxIter(10)
model = kmeans.fit(featureVector)

In [ ]:
predictions = model.transform(featureVector)
predictions.orderBy(['prediction', 'mpg']).show(32)

In [ ]:
from pyspark.ml.evaluation import ClusteringEvaluator


wssse = calculate_wsse(predictions, model)

evaluator = ClusteringEvaluator()
silhouette = evaluator.evaluate(predictions)
print ("k={}, wssse={},  silhouette_score={}".format(k, wssse, silhouette))

## Step 6 : Iterate over K
We are going to calculate WSSSE for various values of K:

In [ ]:
%%time 

from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

kvals = []
silhouettes = []
wssses = []

evaluator = ClusteringEvaluator()

for k in range(2,33):
    kmeans = KMeans().setK(k).setMaxIter(10)
    model = kmeans.fit(featureVector)
    predictions = model.transform(featureVector)
    wssse = calculate_wsse (predictions, model)
    silhouette = evaluator.evaluate(predictions)
    print ("k={}, wssse={},  silhouette_score={}".format(k, wssse, silhouette))
    
    kvals.append(k)
    wssses.append(wssse)
    silhouettes.append(silhouette)

## Step 7 - Plot Silhouette / WSSSE graph

In [ ]:
%matplotlib inline
from matplotlib import pyplot

pyplot.plot(kvals, silhouettes)
pyplot.xlabel('k')
pyplot.ylabel('silhouette score')
pyplot.show()

In [ ]:
pyplot.plot(kvals, wssses)
pyplot.xlabel('k')
pyplot.ylabel('wssse')
pyplot.show()